In [ ]:
import collections
import copy
import cv2
import json
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import scipy.misc
import skimage.exposure

from osgeo import gdal, gdalconst
from pathlib import Path
from scipy import signal
from scipy.ndimage import measurements
from sklearn import linear_model
from typing import Any, Dict, Iterable, List, Mapping, Optional, Tuple

In [ ]:
BandConstants = collections.namedtuple("BandConstants", ["mult", "add", "k1", "k2"])
RADIANCE_ADD_KEY = "RADIANCE_ADD_BAND_"
RADIANCE_MULT_KEY = "RADIANCE_MULT_BAND_"
REFLECTANCE_ADD_KEY = "REFLECTANCE_ADD_BAND_"
REFLECTANCE_MULT_KEY = "REFLECTANCE_MULT_BAND_"
K1_KEY = "K1_CONSTANT_BAND_"
K2_KEY = "K2_CONSTANT_BAND_"


def path_from_filename(landsat_filename):
    split_name = landsat_filename.split("_")
    path, row = split_name[2][:3], split_name[2][3:]
    bands = split_name[0]
    parent_dir = "_".join(split_name[:-1])
    return os.path.join(f"gs://gcp-public-data-landsat/{bands}/01/", path, row, parent_dir, landsat_filename)


def raster_image(tif, downsample_factor=10):
    data = tif.ReadAsArray()
    data = tif.ReadAsArray(
        buf_xsize=int(data.shape[1] / downsample_factor),
        buf_ysize=int(data.shape[0] / downsample_factor),
        resample_alg=gdalconst.GRIORA_Average
    )
    data = np.where(data == 0, np.nan, data)
    return data


def parse_metadata(path_mtl):
    if not os.path.exists(path_mtl):
        raise OSError("Missing file: " + path_mtl)
    metadata = {}
    with open(path_mtl) as f:
        for line in f:
            if " = " in line:
                split_line = line.split(" = ")
                try:
                    metadata[split_line[0].strip()] = float(split_line[-1].strip())
                except ValueError:
                    metadata[split_line[0].strip()] = split_line[-1].strip()
    return metadata


def read_band_constants(metadata, band, reflectance=True):
    if reflectance:
        return BandConstants(
            metadata[REFLECTANCE_MULT_KEY + band],
            metadata[REFLECTANCE_ADD_KEY + band],
            None, 
            None
        )
    else:
        return BandConstants(
            metadata[RADIANCE_MULT_KEY + band],
            metadata[RADIANCE_ADD_KEY + band],
            metadata[K1_KEY + band],
            metadata[K2_KEY + band]
        )


def top_of_atmosphere_temperature(image, band_constants):
    radiance = image * band_constants.mult
    radiance += band_constants.add
    denom = np.log((band_constants.k1 / radiance) + 1.)
    return band_constants.k2 / denom


def reflectance(image, band_constants):
    return image * band_constants.mult + band_constants.add    


def normalize_for_rgb(signal, bounds, adapt=True):
    start, end = bounds
    out = ((signal - start) / (end - start)).clip(0, 1)
    if (np.all(np.isnan(out) | np.isclose(out, 0, atol=1e-3)) or np.all(np.isnan(out) | np.isclose(out, 1, atol=1e-3))):
        adapt = False
    if adapt:
        out = skimage.exposure.equalize_adapthist(out, clip_limit=0.03)
    return out


def truecolor_rgb(path_r, path_g, path_b):
    tif_r = gdal.Open(path_r)
    tif_g = gdal.Open(path_g)
    tif_b = gdal.Open(path_b)
    radiance_r = raster_image(tif_r)
    radiance_g = raster_image(tif_g)
    radiance_b = raster_image(tif_b)

    del tif_r
    del tif_g
    del tif_b
    
    rgb = np.stack([radiance_r, radiance_g, radiance_b], axis=-1)
    rgb = rgb / (2 ** 16 - 1)
    for i in range(3):
        rgb[..., i] = skimage.exposure.equalize_adapthist(rgb[..., i], clip_limit=0.1)
    return rgb


def plot(image, ax):
    ax.imshow(image)
    ax.set_xticks([])
    ax.set_yticks([])


def get_false_color_image(path_11um, path_12um, path_1370nm, path_mtl, night=False):
    tif_11um = gdal.Open(path_11um)
    tif_12um = gdal.Open(path_12um)
    tif_1370nm = gdal.Open(path_1370nm)

    radiance_11um = raster_image(tif_11um)
    radiance_12um = raster_image(tif_12um)
    radiance_1370nm = raster_image(tif_1370nm)

    del tif_11um
    del tif_12um
    del tif_1370nm

    metadata = parse_metadata(path_mtl)
    constants_11um = read_band_constants(metadata, "10", reflectance=False)
    constants_12um = read_band_constants(metadata, "11", reflectance=False)
    constants_1370nm = read_band_constants(metadata, "9", reflectance=True)  

    temperature_11um = top_of_atmosphere_temperature(radiance_11um, constants_11um)
    temperature_12um = top_of_atmosphere_temperature(radiance_12um, constants_12um)
    reflectance_1370nm = reflectance(radiance_1370nm, constants_1370nm)

    tdiff = temperature_11um - temperature_12um

    tdiff_clip = (-1, 5.5)  
    tdiff_bounds = (-tdiff_clip[1], -tdiff_clip[0])
    ir_r = normalize_for_rgb(-1 * tdiff, tdiff_bounds, adapt=False)

    if night:
        ir_g = np.zeros_like(reflectance_1370nm)
        t12_bounds = (243, 303)  # From a sample of nighttime landsat scenes.
    else:
        ir_g = normalize_for_rgb(1 - reflectance_1370nm, bounds=(0.8, 1))
        t12_bounds = (283, 303)  # From a sample of daytime landsat scenes.

    ir_b = normalize_for_rgb(temperature_12um, t12_bounds)
    
    return np.stack([ir_r, ir_g, ir_b], axis=-1), temperature_11um, temperature_12um, reflectance_1370nm

In [ ]:
TWELVE_MICRONS = "temperature_12um"
TDIFF = "brightness_temperature_difference"

def lowpass(image, config):
    kernel_size_pixels = config["lowpass_kernel_size_pixels"]
    pixels_per_sigma = config["lowpass_kernel_pixels_per_sigma"]
    units_per_sigma = pixels_per_sigma / kernel_size_pixels
    n1, n2 = np.meshgrid(
        np.linspace(-1, 1, kernel_size_pixels),
        np.linspace(-1, 1, kernel_size_pixels)
    )
    distance_from_center = np.sqrt(np.square(n1) + np.square(n2))
    gaussian = (1 / (units_per_sigma * np.sqrt(2 * np.pi)) * (np.exp(-np.square(distance_from_center / units_per_sigma) / 2)))
    gaussian_kernel = gaussian / gaussian.sum()
    return signal.convolve2d(image, gaussian_kernel, mode="same")


def synthesize_line_kernel(degrees, config):
    size = config["line_kernel_size_pixels"]
    radians = degrees * (np.pi / 180.)
    n1, n2 = np.meshgrid(np.linspace(-1, 1, size), np.linspace(-1, 1, size))
    d = (n1 * np.cos(radians)) - (n2 * np.sin(radians))
    pixels_per_sigma1 = config["line_kernel_pixels_per_sigma1"]
    pixels_per_sigma2 = (pixels_per_sigma1 + config["line_kernel_pixels_per_sigma_delta"])
    units_per_sigma1 = (pixels_per_sigma1 / config["line_kernel_size_pixels"])
    gaussian1 = ((1 / (units_per_sigma1 * np.sqrt(2 * np.pi))) * (np.exp(-np.square(d / units_per_sigma1) / 2)))
    units_per_sigma2 = (pixels_per_sigma2 / config["line_kernel_size_pixels"])
    gaussian2 = ((1 / (units_per_sigma2 * np.sqrt(2 * np.pi))) * (np.exp(-np.square(d / units_per_sigma2) / 2)))
    difference_of_gaussians = gaussian1 - gaussian2
    return zero_sum_normalize(difference_of_gaussians)


def zero_sum_normalize(kernel: np.ndarray) -> np.ndarray:
    pos_sum = np.sum(np.where(kernel > 0, kernel, 0))
    neg_sum = -np.sum(np.where(kernel <= 0, kernel, 0))
    return np.where(kernel > 0, kernel / pos_sum, kernel / neg_sum)


def get_regional_gradient_mask(
    t_12um: np.ndarray,
    t_12um_stddev: np.ndarray,
    config: Mapping[str, Any]
):
    mean_kernel = np.ones(config["prewitt_operator_size_pixels"], dtype=np.int32)
    gradient_kernel = np.concatenate([
        np.ones(config["prewitt_operator_smoothing_pixels"], dtype=np.int32),
        np.zeros(config["prewitt_operator_size_pixels"] - 2 * config["prewitt_operator_smoothing_pixels"], dtype=np.int32),
        -np.ones(config["prewitt_operator_smoothing_pixels"], dtype=np.int32)
    ])
    prewitt_col = np.outer(mean_kernel, gradient_kernel)
    prewitt_col = zero_sum_normalize(prewitt_col)
    prewitt_row = prewitt_col.T
    gradient_row = signal.convolve2d(t_12um, prewitt_row, mode="same")
    gradient_col = signal.convolve2d(t_12um, prewitt_col, mode="same")
    t_12um_regional_gradient = np.sqrt(np.square(gradient_row) + np.square(gradient_col))
    return (t_12um_regional_gradient < (config["regional_gradient_max_scale"] * t_12um_stddev) + config["regional_gradient_max_offset"])


def mannstein_one_angle_mask(features, config, degrees):
    t_12um = features[TWELVE_MICRONS]
    difference = features[TDIFF]

    t_12um_inverse = -t_12um

    t_12um_inverse_smoothed = lowpass(t_12um_inverse, config)
    difference_smoothed = lowpass(difference, config)

    t_12um_inverse_signal = t_12um_inverse - t_12um_inverse_smoothed
    difference_signal = difference - difference_smoothed

    t_12um_inverse_stddev = lowpass(np.sqrt(np.square(t_12um_inverse_signal)), config)
    difference_stddev = lowpass(np.sqrt(np.square(difference_signal)), config)

    t_12um_inverse_normalized = t_12um_inverse_signal / (t_12um_inverse_stddev + config["stddev_epsilon"])
    difference_normalized = difference_signal / (difference_stddev + config["stddev_epsilon"])

    t_12um_inverse_clipped = np.clip(t_12um_inverse_normalized, -config["normalized_clip_magnitude"], config["normalized_clip_magnitude"])
    difference_clipped = np.clip(difference_normalized, -config["normalized_clip_magnitude"], config["normalized_clip_magnitude"])

    normalized_sum = t_12um_inverse_clipped + difference_clipped

    line_kernel = synthesize_line_kernel(degrees, config)
    detected_lines = signal.convolve2d(normalized_sum, line_kernel, mode='same')

    if "regional_gradient_max_scale" in config:
        regional_gradient_mask = get_regional_gradient_mask(t_12um, t_12um_inverse_stddev, config)
    else:
        regional_gradient_mask = np.ones_like(detected_lines, dtype=np.bool)

    normalized_sum = np.where(np.isfinite(detected_lines), normalized_sum, np.nan)
    difference = np.where(np.isfinite(detected_lines), difference, np.nan)
    regional_gradient_mask = np.where(np.isfinite(detected_lines), regional_gradient_mask, False)

    line_mask = (
        (detected_lines > config["normalized_sum_threshold"]) &
        (normalized_sum > config["normalized_sum_threshold"]) &
        (difference > config["temperature_difference_threshold"]) &
        regional_gradient_mask
    )

    line_mask = np.where(np.isfinite(detected_lines), line_mask, np.nan)
    return line_mask


def get_angles(config):
    return np.linspace(0, 180, config["num_line_kernels"], endpoint=False)


def label_blobs(line_mask, config):
    line_mask = np.where(np.isnan(line_mask), 0, line_mask)
    labels, _ = measurements.label(line_mask, structure=np.ones((3, 3)))
    bincount = np.bincount(labels.flatten())
    right_sized_labels, = (
        (bincount > config["min_contrail_num_pixels"]) &
        (bincount < config["max_contrail_num_pixels"])
    ).nonzero()

    for label in right_sized_labels:
        yield (labels == label).nonzero()


def linear_fit(xs, ys):
    train_ys = ys.reshape(-1, 1)
    train_xs = xs.reshape(-1, 1)
    regression = linear_model.LinearRegression().fit(train_xs, train_ys)
    return (regression.score(train_xs, train_ys), regression.coef_[0][0], regression.intercept_[0])


def linear_and_long_enough(ys, xs, config, avg_size=3, round_to_int=True):
    sorted_indices = np.argsort(ys)
    y1 = np.mean(ys[sorted_indices][:avg_size])
    y2 = np.mean(ys[sorted_indices][-avg_size:])
    sorted_indices = np.argsort(xs)
    x1 = np.mean(xs[sorted_indices][:avg_size])
    x2 = np.mean(xs[sorted_indices][-avg_size:])

    if abs(x1 - x2) > abs(y1 - y2):
        score, slope, intercept = linear_fit(xs=xs, ys=ys)
        y1 = x1 * slope + intercept
        y2 = x2 * slope + intercept
        pixel_length = np.sqrt(np.square(x1 - x2) + np.square(y1 - y2))
    else:
        score, slope, intercept = linear_fit(xs=ys, ys=xs)
        x1 = y1 * slope + intercept
        x2 = y2 * slope + intercept
        pixel_length = np.sqrt(np.square(y1 - y2) + np.square(x1 - x2))

    return (score > np.square(config["min_linear_regression_r_score"]) and pixel_length > config["min_contrail_pixel_length"])


def find_contrail_pixels(line_mask, config):
    contrail_pixel_coords = []
    for rows, cols in label_blobs(line_mask, config):
        if not linear_and_long_enough(rows, cols, config):
            continue
        contrail_pixel_coords.append((rows, cols))
    return contrail_pixel_coords        


def mannstein_contrail_mask(features, config):
    contrail_mask = np.zeros_like(features[TDIFF], dtype=np.int32)
    for degrees in get_angles(config):
        line_mask = mannstein_one_angle_mask(features, config, degrees)
        for rows, cols in find_contrail_pixels(line_mask, config):
            contrail_mask[rows, cols] = 1
    return contrail_mask     

config = {
  "stddev_epsilon": 0.1,
  "normalized_clip_magnitude": 2.0,
  "normalized_sum_threshold": 1,            
  "temperature_difference_threshold": 1.33,
  "max_contrail_num_pixels": 1000,
  "min_contrail_num_pixels": 11,             
  "min_contrail_pixel_length": 14,           
  "min_linear_regression_r_score": 0.25,
  "num_line_kernels": 16,                   
  "line_kernel_size_pixels": 19,
  "line_kernel_pixels_per_sigma1": 1,        
  "line_kernel_pixels_per_sigma_delta": 2, 
  "lowpass_kernel_pixels_per_sigma": 3.88,  
  "lowpass_kernel_size_pixels": 5,
  "prewitt_operator_size_pixels": 15,        
  "prewitt_operator_smoothing_pixels": 3,
  "regional_gradient_max_scale": 2.0,
  "regional_gradient_max_offset": 1.0        
}

In [ ]:
data_dir = "/home/romainlhardy/kaggle/contrails/data/landsat-8/"
dates = ["2021_06_11_1623455786", "2023_01_20_1674247800"]
tmp_dir = "./tmp_data/"
save_dir = "/home/romainlhardy/kaggle/contrails/data/landsat-8/processed"

os.makedirs(tmp_dir, exist_ok=True)
os.makedirs(save_dir, exist_ok=True)

rows = []
for date in dates:
    for i in range(0, 100):
        file_name = os.path.join(data_dir, date, "data", f"landsat_contrails.json-000{i:02d}-of-00100")
        # file_name = "/home/romainlhardy/kaggle/contrails/data/landsat-8/2021_06_11_1623455786/data/demo_shard.json"
        print(file_name)
        with open(file_name, "r") as f:
            for json_obj in f:
                scene_data = json.loads(json_obj)
                filename_11um = scene_data["filename"]
                # filename_11um = "LC08_L1TP_036037_20180406_20180417_01_T1_B10.TIF"
                filename_12um = filename_11um.replace("B10.TIF", "B11.TIF")
                filename_1370nm = filename_11um.replace("B10.TIF", "B9.TIF")
                filename_red = filename_11um.replace("B10.TIF", "B4.TIF")
                filename_green = filename_11um.replace("B10.TIF", "B3.TIF")
                filename_blue = filename_11um.replace("B10.TIF", "B2.TIF")
                filename_mtl = filename_11um.replace("B10.TIF", "MTL.txt")

                !gsutil cp {path_from_filename(filename_11um)} {tmp_dir}
                !gsutil cp {path_from_filename(filename_12um)} {tmp_dir}
                !gsutil cp {path_from_filename(filename_1370nm)} {tmp_dir}
                !gsutil cp {path_from_filename(filename_red)} {tmp_dir}
                !gsutil cp {path_from_filename(filename_green)} {tmp_dir}
                !gsutil cp {path_from_filename(filename_blue)} {tmp_dir}
                !gsutil cp {path_from_filename(filename_mtl)} {tmp_dir}

                path_11um = os.path.join(tmp_dir, filename_11um)
                path_12um = os.path.join(tmp_dir, filename_12um)
                path_1370nm = os.path.join(tmp_dir, filename_1370nm)
                path_red = os.path.join(tmp_dir, filename_red)
                path_green = os.path.join(tmp_dir, filename_green)
                path_blue = os.path.join(tmp_dir, filename_blue)
                path_mtl = os.path.join(tmp_dir, filename_mtl)

                false_color_image, temperature_11um, temperature_12um, reflectance_1370nm = get_false_color_image(path_11um, path_12um, path_1370nm, path_mtl)
                features = {
                    TWELVE_MICRONS: temperature_12um,
                    TDIFF: temperature_11um - temperature_12um,
                }
                mannstein = mannstein_contrail_mask(features, config)   
                mask = np.zeros(false_color_image.shape[:2], dtype=np.uint8)
                patches = []
                for polygon in scene_data["polygons"]:
                    cv2.fillConvexPoly(mask, np.array(polygon).astype(np.int32), 1) 

                hash = random.getrandbits(128)
                img_path = os.path.join(save_dir, "images", f"{hash}.npy")
                msk_path = os.path.join(save_dir, "masks", f"{hash}.npy")
                man_path = os.path.join(save_dir, "mannstein", f"{hash}.npy")

                np.save(img_path, false_color_image)
                np.save(msk_path, mask)
                np.save(man_path, mannstein)

                row = {
                    "record_id": hash,
                    "index": i,
                    "date": date,
                    "image_path": img_path,
                    "mask_path": msk_path,
                    "mannstein_path": man_path
                }
                rows.append(row)

                !rm {path_11um}
                !rm {path_12um}
                !rm {path_1370nm}
                !rm {path_red}
                !rm {path_green}
                !rm {path_blue}
                !rm {path_mtl}

os.rmdir(tmp_dir)

df = pd.DataFrame(rows)
df.to_csv(os.path.join(save_dir, "metadata.csv"), index=None)